In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import time
from Cheby4KANLayer import Cheby4KANLayer

In [2]:
# Define target function
def target_function(x):
    y = np.zeros_like(x)
    mask1 = x < 0.5
    y[mask1] = np.sin(20 * np.pi * x[mask1]) + x[mask1] ** 2
    mask2 = (0.5 <= x) & (x < 1.5)
    y[mask2] = 0.5 * x[mask2] * np.exp(-x[mask2]) + np.abs(np.sin(5 * np.pi * x[mask2]))
    mask3 = x >= 1.5
    y[mask3] = np.log(x[mask3] - 1) / np.log(2) - np.cos(2 * np.pi * x[mask3])

    # add noise
    noise = np.random.normal(0, 0.2, y.shape)
    y += noise
    
    return y

In [3]:
# Define MLP and ChebyKAN
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(1, 128),
            nn.Tanh(),
            nn.Linear(128, 1),
        )

    def forward(self, x):
        return self.layers(x-1) # centralize the input


class Cheby4KAN(nn.Module):
    def __init__(self):
        super(Cheby4KAN, self).__init__()
        self.cheby4kan1 = Cheby4KANLayer(1, 8, 8)
        self.cheby4kan2 = Cheby4KANLayer(8, 1, 8)

    def forward(self, x):
        x = self.cheby4kan1(x)
        x = self.cheby4kan2(x)
        return x

In [5]:
# Generate sample data
x_train = torch.linspace(0, 2, steps=500).unsqueeze(1)
y_train = torch.tensor(target_function(x_train))

# Instantiate models
cheby4_model = Cheby4KAN()
mlp_model = SimpleMLP()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer_cheby4 = torch.optim.Adam(cheby4_model.parameters(), lr=0.01)
optimizer_mlp = torch.optim.Adam(mlp_model.parameters(), lr=0.03)

cheby4_losses = []
mlp_losses = []

# Train the models
epochs = 200000
for epoch in range(epochs):
    optimizer_cheby4.zero_grad()
    outputs_cheby4 = cheby4_model(x_train)
    loss_cheby4 = criterion(outputs_cheby4, y_train)
    loss_cheby4.backward()
    optimizer_cheby4.step()

    optimizer_mlp.zero_grad()
    outputs_mlp = mlp_model(x_train)
    loss_mlp = criterion(outputs_mlp, y_train)
    loss_mlp.backward()
    optimizer_mlp.step()

    if epoch % 100 == 0:
        cheby4_losses.append(loss_cheby4.item())
        mlp_losses.append(loss_mlp.item())
        print(f'Epoch {epoch + 1}/{epochs}, Cheby4KAN Loss: {loss_cheby4.item():.4f}, MLP Loss: {loss_mlp.item():.4f}')

# Test the models
x_test = torch.linspace(0, 2, steps=400).unsqueeze(1)
y_pred_cheby4 = cheby4_model(x_test).detach()
y_pred_mlp = mlp_model(x_test).detach()

RuntimeError: unsupported operation: more than one element of the written-to tensor refers to a single memory location. Please clone() the tensor before performing the operation.